In [2]:
#!/usr/bin/env python3
"""
Env info collector (no expected checks).
Prints Python/OS/CPU/RAM/GPU details for verification documentation.
"""

import os
import platform
import shutil
import subprocess
import sys
from datetime import datetime

def run(cmd):
    """Run a command and return (ok, output_str)."""
    try:
        p = subprocess.run(cmd, capture_output=True, text=True, check=False)
        out = (p.stdout or "").strip()
        err = (p.stderr or "").strip()
        ok = (p.returncode == 0)
        return ok, out if out else err
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

def bytes_to_gib(n):
    return n / (1024 ** 3)

def print_kv(k, v):
    print(f"{k:20s}: {v}")

def main():
    print("=== Environment Info (for verification) ===")
    print_kv("Collected at", datetime.now().isoformat(timespec="seconds"))

    # Python
    print("\n--- Python ---")
    print_kv("Executable", sys.executable)
    print_kv("Version", sys.version.replace("\n", " "))
    print_kv("Implementation", platform.python_implementation())

    # OS / distro
    print("\n--- OS ---")
    print_kv("System", platform.system())
    print_kv("Release", platform.release())
    print_kv("Version", platform.version())
    print_kv("Machine", platform.machine())
    print_kv("Platform", platform.platform())

    # Try to read /etc/os-release (Ubuntu version details)
    if os.path.exists("/etc/os-release"):
        try:
            with open("/etc/os-release", "r", encoding="utf-8") as f:
                data = f.read().splitlines()
            kv = {}
            for line in data:
                if "=" in line and not line.startswith("#"):
                    a, b = line.split("=", 1)
                    kv[a.strip()] = b.strip().strip('"')
            if kv:
                print_kv("Distro NAME", kv.get("NAME", ""))
                print_kv("Distro VERSION", kv.get("VERSION", ""))
                print_kv("Distro VERSION_ID", kv.get("VERSION_ID", ""))
        except Exception as e:
            print_kv("/etc/os-release", f"Error: {e}")

    # CPU
    print("\n--- CPU ---")
    print_kv("Processor (platform)", platform.processor() or "(empty)")
    print_kv("Logical cores", os.cpu_count())

    # Best-effort CPU model on Linux
    if os.path.exists("/proc/cpuinfo"):
        try:
            model = None
            with open("/proc/cpuinfo", "r", encoding="utf-8") as f:
                for line in f:
                    if line.lower().startswith("model name"):
                        model = line.split(":", 1)[1].strip()
                        break
            print_kv("CPU Model (/proc)", model or "Not found")
        except Exception as e:
            print_kv("CPU Model (/proc)", f"Error: {e}")

    # RAM (via psutil if available; fallback to /proc/meminfo)
    print("\n--- Memory (RAM) ---")
    try:
        import psutil  # pip install psutil
        vm = psutil.virtual_memory()
        print_kv("Total", f"{bytes_to_gib(vm.total):.2f} GiB")
        print_kv("Available", f"{bytes_to_gib(vm.available):.2f} GiB")
        print_kv("Used", f"{bytes_to_gib(vm.used):.2f} GiB")
    except Exception:
        # Fallback: /proc/meminfo
        if os.path.exists("/proc/meminfo"):
            try:
                memtotal_kib = None
                with open("/proc/meminfo", "r", encoding="utf-8") as f:
                    for line in f:
                        if line.startswith("MemTotal:"):
                            memtotal_kib = int(line.split()[1])
                            break
                if memtotal_kib is not None:
                    gib = (memtotal_kib * 1024) / (1024 ** 3)
                    print_kv("Total", f"{gib:.2f} GiB (from /proc/meminfo)")
                else:
                    print_kv("Total", "Not found in /proc/meminfo")
            except Exception as e:
                print_kv("Memory", f"Error: {e}")
        else:
            print_kv("Memory", "psutil not installed and /proc/meminfo not found")

    # GPU (NVIDIA)
    print("\n--- GPU (NVIDIA) ---")
    nvsmi = shutil.which("nvidia-smi")
    if not nvsmi:
        print_kv("nvidia-smi", "Not found (NVIDIA driver/tool not installed or not in PATH)")
    else:
        print_kv("nvidia-smi", nvsmi)

        # Detailed query
        ok, out = run([
            "nvidia-smi",
            "--query-gpu=name,driver_version,vbios_version,memory.total,compute_cap,pci.bus_id",
            "--format=csv,noheader"
        ])
        if ok:
            print_kv("GPU Query", out)
        else:
            print_kv("GPU Query", f"Failed: {out}")

        # Full summary (optional, but useful for logs)
        ok2, out2 = run(["nvidia-smi"])
        if ok2:
            print("\n[nvidia-smi full output]")
            print(out2)
        else:
            print_kv("nvidia-smi", f"Failed: {out2}")

    print("\n=== Done ===")

if __name__ == "__main__":
    main()


=== Environment Info (for verification) ===
Collected at        : 2026-02-03T16:56:56

--- Python ---
Executable          : e:\ney\envs\MSTGNet\python.exe
Version             : 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
Implementation      : CPython

--- OS ---
System              : Windows
Release             : 10
Version             : 10.0.26100
Machine             : AMD64
Platform            : Windows-10-10.0.26100-SP0

--- CPU ---
Processor (platform): AMD64 Family 25 Model 33 Stepping 2, AuthenticAMD
Logical cores       : 16

--- Memory (RAM) ---
Total               : 63.93 GiB
Available           : 50.42 GiB
Used                : 13.51 GiB

--- GPU (NVIDIA) ---
nvidia-smi          : C:\WINDOWS\system32\nvidia-smi.EXE
GPU Query           : NVIDIA GeForce RTX 3060, 560.94, 94.06.25.00.87, 12288 MiB, 8.6, 00000000:2B:00.0

[nvidia-smi full output]
Tue Feb  3 16:56:56 2026       
+--------------------------------------------------